### GPT -3 란?
GPT -3란 open AI 사에서 개발한 인공지능 모델로, 무려 3000억 개의 데이터 셋으로 학습을 하였으며 1750개의 매개변수를 가지고 있다.

지금까지 언어모델은 Pretrain을 통해 학습한 후 많진 않지만 적지 않은 꽤 많은 데이터를 수집하여 fine-tuning을 해서 언어모델의 parameter weights 값들을 업데이트해야 동작했다.

하지만 GPT3 모델은 weight 업데이트 없이 언어모델의 입력인 prompt에 정보를 잘 담으면 학습 없이도 많은 task가 가능하다.

GPT3로 할수있는 task는 자연어처리의 모든 task이다. 분류/요약/번역/챗봇 등등 수많은 task가 학습 없이 prompt 입력 만으로 가능하다.

- Reference: https://keep-steady.tistory.com/51
- OpenAI API key 요청: https://beta.openai.com/account/api-keys
- GPT3 사용 예시: https://beta.openai.com/playground
- 텍스트  입력:  가운데 텍스트 입력창에 텍스트를 입력한다. 예시는 요약 예시이므로 긴 원문을 입력하였다.
- 파라미터 설정: 오른쪽 파라미터 설정 란에서 temperature, maximum length, TopP, Penalty 등의 파라미터를 입력한다.Maximum length는 최대 생성 길이로, GPT3는 2048 subword token만큼의 출력을 생성할 수 있다.Temperature와 TopP는 글 생성 시 샘플링 방법이다. 이 방식에 따라 출력이 달라진다.
- 제출 및 결과 출력: prompt를 제출하면, 응답으로 answer가 생성되어 출력된다.
- GPT-3 예시: OpenAI에선 GPT3로 할 수 있는 많은 예시들을 보여준다. 클릭하면 Playground에서 사용할 수 있는 예시가 열린다. QA, 요약, 번역, 분류 등등 수많은 모든 자연어처리 task를 손쉽게 사용할 수 있다.아래 링크에서 확인할 수 있다.
- https://beta.openai.com/examples



**1. openai 설치

코랩을 켜고 다음과 같은 명령어로 openai를 설치한다. **

In [ ]:
!pip install openai

**2. OpenAI key 설정**

In [ ]:
My_OpenAI_key = 'Your_Open_AI_Key'

import openai
openai.api_key = My_OpenAI_key
completion = openai.Completion()

- 참고: 하이퍼 파라미터 (Reference: https://crossroads.tistory.com/11)
 - 온도(Temperature): 창의성 제어 => 온도가 높으면 결과물이 더 다양하고 예측할 수 없는 결과가 나오는 반면, 온도가 낮으면 결과물이 더 보수적이고 예측 가능하다.
  - 빔 폭(Beam Width): ChatGPT에 의해 생성되는 연속적인 결과물의 수를 제어한다. 빔 폭이 크면 결과물 시퀀스가 많아지고 고품질의 결과물을 생성할 가능성이 높아지지만 응답을 생성하는데 필요한 시간과 리소스도 증가한다. 빔 폭이 낮으면 결과물 시퀀스가 더 적게 생성되지만 더 빠르고 리소스가 더 적게 필요하다.
  - Top-p: 결과물 시퀀스에서 다음 단어를 선택하는 데 사용되는 확률 임계값을 제어한다. Top-p 값이 높으면 더 큰 단어 풀에서 선택되므로 결과물이 더 다양하고 예측할 수 없게 된다. 반면 Top-p 값이 낮으면 단어 풀이 더 작아지므로 결과물이 더 보수적이고 예측 가능해집니다.
 - 길이 패널티(Length Penalty): 결과물의 길이와 품질 사이의 균형을 제어한다. 길이 패널티를 높게 설정하면 결과물 시퀀스가 짧아지고, 낮게 설정하면 결과물 시퀀스가 길어진다.
 **3. GPT3 config 지정**
- GPT3를 돌리기 위한 config를 지정한다. 이때 max_tokens으로 생성될 글의 길이를, temperature와 top_p로 생성될 다음 단어의 샘플링 방법 등을 지정한다.

In [ ]:
# config
temperature = 0.9
max_tokens =50
top_p = 1.0
best_of = 1
frequency_penalty = 0.0
presence_penalty = 0.0

# stop = ["You:"]
stop = ["\n"]

**3. 챗봇 테스트**
- openai CLI에서 API로 통신이 잘 되는지 테스트해 보자. 질문을 'What is your name?'으로 줬을 때 응답이 잘 오는지 테스트해본다.

In [ ]:
# chatbot test
question = ' what is your name?'
prompt_initial = f'Human:%s\nAI: ' % (question)

prompt = prompt_initial

response = completion.create(
    prompt=prompt,
    engine="davinci",
    max_tokens=max_tokens,
    stop=stop,
    temperature=temperature,
    top_p=top_p,
    best_of=best_of,
)
answer = response.choices[0].text.strip()
print(prompt, answer)

**4. 챗봇 구축**
- 3에서 응답이 잘 오는 것을 확인했으면, 이제 본격적으로 챗봇을 만들자.
- question에 질문을 입력하면 history에서 주고받는 대화를 기록하여 지난 대화 기반으로 응답을 만들어 주도록 prompt를 설계하였다.

In [ ]:
def run_openai_chatbot(question='What time is it?', history=''):

    prompt_initial = f'Human:%s\nAI:' % (question)

    prompt = history + '\n' + prompt_initial

    response = completion.create(
        prompt=prompt,
        engine="davinci",
        max_tokens=max_tokens,
        stop=stop,
        temperature=temperature,
        top_p=top_p,
        best_of=best_of,
    )
    answer = response.choices[0].text.strip()
    history = prompt + answer

    print('question: %s\nanswer: %s\n\nhistory: %s' % (question, answer, history))
    return answer, history

**5. 챗봇 실습(1질문, 1답변)**
- 4에서 만든 함수를 이용하여 question에 질문을 입력하면 answer를 생성하는 함수로 답변을 만든다.
- 이때 history는 과거에 주고받은 대화를 기록하여 다음 대화 때 사용하기 위해 저장한다.



In [ ]:
question='What time is it?'
answer, history = run_openai_chatbot(question=question, history='')

In [ ]:
question='Really?'
answer, history = run_openai_chatbot(question=question, history=history)

In [ ]:
question='I am hungry...'
answer, history = run_openai_chatbot(question=question, history=history)

In [ ]:
question='I want to have a lunch'
answer, history = run_openai_chatbot(question=question, history=history)

**6. 챗봇 실습(주고받기)**

In [ ]:
for turn in range(3):
    question = input(">> User:")
    answer, history = run_openai_chatbot(question=question, history=history)